## Logic for calculating the false negative rate

#### Case 1 repeats (8 experiments, 1000 repeats each):

- None of the structures are actually affected, so all of the structures are true negatives; in this case, it's impossible to have false negatives $\rightarrow$ skip

#### Case 2 repeats (8 experiments, 1000 repeats each):

- Left hippocampus (levels 1&2) are actually affected 
- Record how many times any structure (excluding left amygdala levels 1 and 2) has p<0.05 
- Use 6 counters $\rightarrow$ produce 6 false negative rates

#### Case 3 repeats (8 experiments, 1000 repeats each):

- Left hippocampus (levels 1&2) and left amygdala (levels 1&2) are actually affected 
- Record how many times any structure has p<0.05 
- Use 8 counters $\rightarrow$ produce 8 false negative rates  

#### Case 4 repeats (8 experiments, 1000 repeats each):

- Calculate false negative rate in the same way that we calculate false negative rate for case 3

#### In total, there are 8000 * 4 = 32,000 repeats

## Setup

### Load packages

In [16]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd

import scipy
from scipy import stats
from scipy.stats import binom

### Define the sorting function

In [17]:
dictionary = {"Everything" : 1, 
              "Telencephalon_L_501_5" : 2,
              "CerebralCortex_L_482_4": 3, 
              "Limbic_L_434_3" : 4,
              "Hippo_L_338_2" : 5,
              "Amyg_L_336_2" : 6,
              "Hippo_L_75_1" : 7,
              "Amyg_L_73_1" : 8
             }


# Purpose: to sort structures that have ties in probability of being affected P[Z=1|X]. 
    # Tied structures are sorted from parent to child
# Input: a list of strings from one .npz file
    # Each list contains the structure name, probability of being affected, and the p-value
# Output: sorted list of strings according to the rules above

def sorting_function(input_string):
    my_list = input_string
    my_list = sorted(my_list, key = lambda x : dictionary[x.split(",")[0]])  
    my_list = sorted(my_list, key = lambda x : float(x.split(",")[1].split("=")[-1]), reverse = True) 
    return my_list

## Case 2 repeats

General format of case 2 repeat files: case_2_experiment_x_repeat_xxxx.npz

Case 2 experiments 1-8, repeats 0000 to 0999 → 8 * 1000 = 8000 files

### Case 2: Obtaining a list of file names

In [3]:
case_2 = []

for i in range(1, 9):
    for j in range(0, 1000):
        if j < 10:
            string = "fwer_experiments/case_2_experiment_" + str(i) + "_repeat_" + "000" + str(j) + ".npz"
        elif j < 100:
            string = "fwer_experiments/case_2_experiment_" + str(i) + "_repeat_" + "00" + str(j) + ".npz"
        else:
            string = "fwer_experiments/case_2_experiment_" + str(i) + "_repeat_" + "0" + str(j) + ".npz"
        case_2.append(string)

case_2[0:4], case_2[7996:8001]

(['fwer_experiments/case_2_experiment_1_repeat_0000.npz',
  'fwer_experiments/case_2_experiment_1_repeat_0001.npz',
  'fwer_experiments/case_2_experiment_1_repeat_0002.npz',
  'fwer_experiments/case_2_experiment_1_repeat_0003.npz'],
 ['fwer_experiments/case_2_experiment_8_repeat_0996.npz',
  'fwer_experiments/case_2_experiment_8_repeat_0997.npz',
  'fwer_experiments/case_2_experiment_8_repeat_0998.npz',
  'fwer_experiments/case_2_experiment_8_repeat_0999.npz'])

### Case 2 experiment 1: Calculating the false negatives rate

Case 2: left hippocampus is affected

Experiment 1: clip the probabilities at 0.999 and 0.001 (original)

In [4]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0

for i in range(0, 1000): # For each of the 1000 case 2 experiment 1 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_2[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break


print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)

Everything: 0.010000000000000009
Telencephalon: 0.010000000000000009
Cerebral cortex: 0.010000000000000009
Limbic system: 0.010000000000000009
Hippocampus level 2: 0.010000000000000009
Hippocampus level 1: 0.010000000000000009


### Case 2 experiment 2: Calculating the false negative rate

Case 2: left hippocampus is affected

Experiment 2: clip the probabilities at 0.9999 and 0.0001

In [5]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0

for i in range(1000, 2000): # For each of the 100 case 2 experiment 2 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_2[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break


print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)

Everything: 0.01200000000000001
Telencephalon: 0.01200000000000001
Cerebral cortex: 0.01200000000000001
Limbic system: 0.01200000000000001
Hippocampus level 2: 0.01200000000000001
Hippocampus level 1: 0.01200000000000001


### Case 2 experiment 3: Calculating the false negative rate

Case 2: left hippocampus is affected

Experiment 3: run 20 iterations of the EM algorithm (original)

In [6]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0

for i in range(2000, 3000): # For each of the 100 case 2 experiment 3 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_2[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For the last two structures (left amygdala levels 1 and 2) in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break


print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)

Everything: 0.007000000000000006
Telencephalon: 0.007000000000000006
Cerebral cortex: 0.007000000000000006
Limbic system: 0.007000000000000006
Hippocampus level 2: 0.007000000000000006
Hippocampus level 1: 0.007000000000000006


### Case 2 experiment 4: Calculating the false negative rate

Case 2: left hippocampus is affected

Experiment 4: run 50 iterations of the EM algorithm

In [7]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0

for i in range(3000, 4000): # For each of the 1000 case 2 experiment 4 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_2[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break


print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)

Everything: 0.016000000000000014
Telencephalon: 0.016000000000000014
Cerebral cortex: 0.016000000000000014
Limbic system: 0.016000000000000014
Hippocampus level 2: 0.016000000000000014
Hippocampus level 1: 0.016000000000000014


### Case 2 experiment 5: Calculating the false negative rate

Case 2: left hippocampus is affected

Experiment 5: run 100 iterations of the EM algorithm

In [8]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0

for i in range(4000, 5000): # For each of the 1000 case 2 experiment 5 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_2[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break


print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)

Everything: 0.015000000000000013
Telencephalon: 0.015000000000000013
Cerebral cortex: 0.015000000000000013
Limbic system: 0.015000000000000013
Hippocampus level 2: 0.015000000000000013
Hippocampus level 1: 0.015000000000000013


### Case 2 experiment 6: Calculating the false negative rate

Case 2: left hippocampus is affected

Experiment 6: initial probabilities are 0.5 (original)

In [9]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0

for i in range(5000, 6000): # For each of the 1000 case 2 experiment 6 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_2[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break


print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)

Everything: 0.01200000000000001
Telencephalon: 0.01200000000000001
Cerebral cortex: 0.01200000000000001
Limbic system: 0.01200000000000001
Hippocampus level 2: 0.01200000000000001
Hippocampus level 1: 0.01200000000000001


### Case 2 experiment 7: Calculating the false negative rate

Case 2: left hippocampus is affected

Experiment 7: initial probabilities are 0.25

In [10]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0

for i in range(6000, 7000): # For each of the 1000 case 2 experiment 7 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_2[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break


print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)

Everything: 0.009000000000000008
Telencephalon: 0.009000000000000008
Cerebral cortex: 0.009000000000000008
Limbic system: 0.009000000000000008
Hippocampus level 2: 0.009000000000000008
Hippocampus level 1: 0.009000000000000008


### Case 2 experiment 8: Calculating the false negative rate

Case 2: left hippocampus is affected

Experiment 8: initial probabilities are 0.75

In [11]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0

for i in range(7000, 8000): # For each of the 1000 case 2 experiment 8 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_2[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break


print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)

Everything: 0.01200000000000001
Telencephalon: 0.01200000000000001
Cerebral cortex: 0.01200000000000001
Limbic system: 0.01200000000000001
Hippocampus level 2: 0.01200000000000001
Hippocampus level 1: 0.01200000000000001


## Case 3 repeats

General format of case 3 repeat files: case_3_experiment_x_repeat_xxxx.npz

Case 3 experiments 1-8, repeats 0000 to 0999 → 8 * 1000 = 8000 files

In [3]:
case_3 = []

for i in range(1, 9):
    for j in range(0, 1000):
        if j < 10:
            string = "fwer_experiments/case_3_experiment_" + str(i) + "_repeat_" + "000" + str(j) + ".npz"
        elif j < 100:
            string = "fwer_experiments/case_3_experiment_" + str(i) + "_repeat_" + "00" + str(j) + ".npz"
        else:
            string = "fwer_experiments/case_3_experiment_" + str(i) + "_repeat_" + "0" + str(j) + ".npz"
        case_3.append(string)

case_3[0:4], case_3[7996:8001]

(['fwer_experiments/case_3_experiment_1_repeat_0000.npz',
  'fwer_experiments/case_3_experiment_1_repeat_0001.npz',
  'fwer_experiments/case_3_experiment_1_repeat_0002.npz',
  'fwer_experiments/case_3_experiment_1_repeat_0003.npz'],
 ['fwer_experiments/case_3_experiment_8_repeat_0996.npz',
  'fwer_experiments/case_3_experiment_8_repeat_0997.npz',
  'fwer_experiments/case_3_experiment_8_repeat_0998.npz',
  'fwer_experiments/case_3_experiment_8_repeat_0999.npz'])

### Case 3 experiment 1: Calculating the false negatives rate

Case 3: both are affected

Experiment 1: clip the probabilities at 0.999 and 0.001 (original)

In [4]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(0, 1000): # For each of the 1000 case 3 experiment 1 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_3[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0
Telencephalon: 0.0
Cerebral cortex: 0.0
Limbic system: 0.0
Hippocampus level 2: 0.0
Hippocampus level 1: 0.489
Amygdala level 2: 0.0
Amygdala level 1: 0.511


### Case 3 experiment 2: Calculating the false negatives rate

Case 3: both are affected

Experiment 2: clip the probabilities at 0.9999 and 0.0001

In [5]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(1000, 2000): # For each of the 1000 case 3 experiment 2 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_3[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0
Telencephalon: 0.0
Cerebral cortex: 0.0
Limbic system: 0.0
Hippocampus level 2: 0.0
Hippocampus level 1: 0.518
Amygdala level 2: 0.0
Amygdala level 1: 0.482


### Case 3 experiment 3: Calculating the false negatives rate

Case 3: both are affected

Experiment 3: run 20 iterations of the EM algorithm (original)

In [6]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(2000, 3000): # For each of the 1000 case 3 experiment 3 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_3[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0
Telencephalon: 0.0
Cerebral cortex: 0.0
Limbic system: 0.0
Hippocampus level 2: 0.0
Hippocampus level 1: 0.516
Amygdala level 2: 0.0
Amygdala level 1: 0.484


### Case 3 experiment 4: Calculating the false negatives rate

Case 3: both are affected

Experiment 4: run 50 iterations of the EM algorithm

In [7]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(3000, 4000): # For each of the 1000 case 3 experiment 4 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_3[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0
Telencephalon: 0.0
Cerebral cortex: 0.0
Limbic system: 0.0
Hippocampus level 2: 0.0
Hippocampus level 1: 0.498
Amygdala level 2: 0.0
Amygdala level 1: 0.502


### Case 3 experiment 5: Calculating the false negatives rate

Case 3: both are affected

Experiment 5: run 100 iterations of the EM algorithm

In [8]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(4000, 5000): # For each of the 1000 case 3 experiment 5 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_3[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0
Telencephalon: 0.0
Cerebral cortex: 0.0
Limbic system: 0.0
Hippocampus level 2: 0.0
Hippocampus level 1: 0.531
Amygdala level 2: 0.0
Amygdala level 1: 0.469


### Case 3 experiment 6: Calculating the false negatives rate

Case 3: both are affected

Experiment 6: initial probabilities are 0.5 (original)

In [9]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(5000, 6000): # For each of the 1000 case 3 experiment 6 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_3[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0
Telencephalon: 0.0
Cerebral cortex: 0.0
Limbic system: 0.0
Hippocampus level 2: 0.0
Hippocampus level 1: 0.503
Amygdala level 2: 0.0
Amygdala level 1: 0.497


### Case 3 experiment 7: Calculating the false negatives rate

Case 3: both are affected

Experiment 7: initial probabilities are 0.25

In [10]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(6000, 7000): # For each of the 1000 case 3 experiment 7 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_3[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0
Telencephalon: 0.0
Cerebral cortex: 0.0
Limbic system: 0.0
Hippocampus level 2: 0.0
Hippocampus level 1: 0.475
Amygdala level 2: 0.0
Amygdala level 1: 0.525


### Case 3 experiment 8: Calculating the false negatives rate

Case 3: both are affected

Experiment 8: initial probabilities are 0.75

In [11]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(7000, 8000): # For each of the 1000 case 3 experiment 8 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_3[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0
Telencephalon: 0.0
Cerebral cortex: 0.0
Limbic system: 0.0
Hippocampus level 2: 0.0
Hippocampus level 1: 0.504
Amygdala level 2: 0.0
Amygdala level 1: 0.496


## Case 4 repeats

General format of case 4 repeat files: case_4_experiment_x_repeat_xxxx.npz

Case 4 experiments 1-8, repeats 0000 to 0999 → 8 * 1000 = 8000 files

In [12]:
case_4 = []

for i in range(1, 9):
    for j in range(0, 1000):
        if j < 10:
            string = "fwer_experiments/case_4_experiment_" + str(i) + "_repeat_" + "000" + str(j) + ".npz"
        elif j < 100:
            string = "fwer_experiments/case_4_experiment_" + str(i) + "_repeat_" + "00" + str(j) + ".npz"
        else:
            string = "fwer_experiments/case_4_experiment_" + str(i) + "_repeat_" + "0" + str(j) + ".npz"
        case_4.append(string)

case_4[0:4], case_4[7996:8001]

(['fwer_experiments/case_4_experiment_1_repeat_0000.npz',
  'fwer_experiments/case_4_experiment_1_repeat_0001.npz',
  'fwer_experiments/case_4_experiment_1_repeat_0002.npz',
  'fwer_experiments/case_4_experiment_1_repeat_0003.npz'],
 ['fwer_experiments/case_4_experiment_8_repeat_0996.npz',
  'fwer_experiments/case_4_experiment_8_repeat_0997.npz',
  'fwer_experiments/case_4_experiment_8_repeat_0998.npz',
  'fwer_experiments/case_4_experiment_8_repeat_0999.npz'])

### Case 4 experiment 1: Calculating the false negatives rate

Case 4: either left hippocampus or left amygdala is affected but not both

Experiment 1: clip the probabilities at 0.999 and 0.001 (original)

In [13]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(0, 1000): # For each of the 1000 case 4 experiment 1 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_4[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0
Telencephalon: 0.0
Cerebral cortex: 0.0
Limbic system: 0.0
Hippocampus level 2: 0.14
Hippocampus level 1: 0.502
Amygdala level 2: 0.14500000000000002
Amygdala level 1: 0.552


### Case 4 experiment 2: Calculating the false negatives rate

Case 4: either left hippocampus or left amygdala is affected but not both

Experiment 2: clip the probabilities at 0.9999 and 0.0001

In [14]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(1000, 2000): # For each of the 1000 case 4 experiment 2 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_4[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0010000000000000009
Telencephalon: 0.0010000000000000009
Cerebral cortex: 0.0010000000000000009
Limbic system: 0.0010000000000000009
Hippocampus level 2: 0.14300000000000002
Hippocampus level 1: 0.511
Amygdala level 2: 0.137
Amygdala level 1: 0.544


### Case 4 experiment 3: Calculating the false negatives rate

Case 4: either left hippocampus or left amygdala is affected but not both

Experiment 3: run 20 iterations of the EM algorithm (original)

In [15]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(2000, 3000): # For each of the 1000 case 4 experiment 3 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_4[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0010000000000000009
Telencephalon: 0.0010000000000000009
Cerebral cortex: 0.0010000000000000009
Limbic system: 0.0010000000000000009
Hippocampus level 2: 0.135
Hippocampus level 1: 0.524
Amygdala level 2: 0.15000000000000002
Amygdala level 1: 0.5449999999999999


### Case 4 experiment 4: Calculating the false negatives rate

Case 4: either left hippocampus or left amygdala is affected but not both

Experiment 4: run 50 iterations of the EM algorithm

In [18]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(3000, 4000): # For each of the 1000 case 4 experiment 4 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_4[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0050000000000000044
Telencephalon: 0.0050000000000000044
Cerebral cortex: 0.0050000000000000044
Limbic system: 0.0050000000000000044
Hippocampus level 2: 0.134
Hippocampus level 1: 0.521
Amygdala level 2: 0.137
Amygdala level 1: 0.5409999999999999


### Case 4 experiment 5: Calculating the false negatives rate

Case 4: either left hippocampus or left amygdala is affected but not both

Experiment 5: run 100 iterations of the EM algorithm

In [19]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(4000, 5000): # For each of the 1000 case 4 experiment 5 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_4[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0040000000000000036
Telencephalon: 0.0040000000000000036
Cerebral cortex: 0.0040000000000000036
Limbic system: 0.0040000000000000036
Hippocampus level 2: 0.15000000000000002
Hippocampus level 1: 0.5609999999999999
Amygdala level 2: 0.135
Amygdala level 1: 0.495


### Case 4 experiment 6: Calculating the false negatives rate

Case 4: either left hippocampus or left amygdala is affected but not both

Experiment 6: initial probabilities are 0.5 (original)

In [20]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(5000, 6000): # For each of the 1000 case 4 experiment 6 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_4[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0020000000000000018
Telencephalon: 0.0020000000000000018
Cerebral cortex: 0.0020000000000000018
Limbic system: 0.0020000000000000018
Hippocampus level 2: 0.14100000000000001
Hippocampus level 1: 0.527
Amygdala level 2: 0.14600000000000002
Amygdala level 1: 0.527


### Case 4 experiment 7: Calculating the false negatives rate

Case 4: either left hippocampus or left amygdala is affected but not both

Experiment 7: initial probabilities are 0.25

In [21]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(6000, 7000): # For each of the 1000 case 4 experiment 7 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_4[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0040000000000000036
Telencephalon: 0.0040000000000000036
Cerebral cortex: 0.0040000000000000036
Limbic system: 0.0040000000000000036
Hippocampus level 2: 0.16100000000000003
Hippocampus level 1: 0.532
Amygdala level 2: 0.15400000000000003
Amygdala level 1: 0.5369999999999999


### Case 4 experiment 8: Calculating the false negatives rate

Case 4: either left hippocampus or left amygdala is affected but not both

Experiment 8: initial probabilities are 0.75

In [22]:
count_everything = 0
count_telencephalon = 0
count_cerebral_cortex = 0
count_limbic = 0
count_hippo2 = 0
count_hippo1 = 0
count_amyg2 = 0
count_amyg1 = 0

for i in range(7000, 8000): # For each of the 1000 case 4 experiment 8 files...
    reject_p = np.array([0, 0, 0, 0, 0, 0, 0, 0]) # For each repeat, assume no structures are affected (null hypothesis)
    file = np.load(case_4[i]) # Load the file
    file = file["arr_0"] # Values corresponding to the key
    file = sorting_function(file)
    
    for j in range(0, 7): # For each structure in each repeat...
        p = file[j].find("p=") # Index of "p" in the string
        pval = float(file[j][p::][2::]) # Extract the p-value
        if pval < 0.05: # If the p-value is < 0.05...
            reject_p[j] = 1
        else:
            break
    
    for r, s in zip(reject_p, file):
        if "Everything" in s and r: # If "Everything" is in the string and we decided to reject H0...
            count_everything += 1
        elif "Telencephalon" in s and r: # If "Telencephalon" is in the string and we decided to reject H0...
            count_telencephalon += 1
        elif "CerebralCortex" in s and r: # If "CerebralCortex" is in the string and we decided to reject H0...
            count_cerebral_cortex += 1
        elif "Limbic" in s and r: # If "Limbic" is in the string and we decided to reject H0...
            count_limbic += 1
        elif "Hippo_L_338_2" in s and r: # If "Hippo_L_338_2" is in the string and we decided to reject H0...
            count_hippo2 += 1
        elif "Hippo_L_75_1" in s and r: # If "Hippo_L_75_1" is in the string and we decided to reject H0...
            count_hippo1 += 1
        elif "Amyg_L_336_2" in s and r: # If "Amyg_L_336_2" is in the string and we decided to reject H0...
            count_amyg2 += 1
        elif "Amyg_L_73_1" in s and r: # If "Amyg_L_73_1" is in the string and we decided to reject H0...
            count_amyg1 += 1
        elif not r: # As soon as we fail to reject one structure, stop
            break

print("Everything:", 1 - count_everything / 1000)
print("Telencephalon:", 1 - count_telencephalon / 1000)
print("Cerebral cortex:", 1 - count_cerebral_cortex / 1000)
print("Limbic system:", 1 - count_limbic / 1000)
print("Hippocampus level 2:", 1 - count_hippo2 / 1000)
print("Hippocampus level 1:", 1 - count_hippo1 / 1000)
print("Amygdala level 2:", 1 - count_amyg2 / 1000)
print("Amygdala level 1:", 1 - count_amyg1 / 1000)

Everything: 0.0010000000000000009
Telencephalon: 0.0010000000000000009
Cerebral cortex: 0.0010000000000000009
Limbic system: 0.0010000000000000009
Hippocampus level 2: 0.134
Hippocampus level 1: 0.521
Amygdala level 2: 0.14900000000000002
Amygdala level 1: 0.5369999999999999
